In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

cache_dir = "/data4/yoomcache"
model_cache_dir = os.path.join(cache_dir, 'huggingface')
data_cache_dir = os.path.join(cache_dir, 'datasets')
checkpoint_dir = os.path.join(cache_dir, 'checkpoint')

import torch
from datasets import load_dataset, load_metric
import math
from itertools import groupby

import wandb
wandb.init(project="testing-wav2vec2gpt", entity="yoom-private")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yoom-private (use `wandb login --relogin` to force relogin)


In [2]:
# %reload_ext autoreload
# %autoreload 2
from wav2vec2GPTwCTC import *
from configuration_wav2vec2gpt import Wav2Vec2GPTConfig

from transformers import Wav2Vec2FeatureExtractor
from transformers import GPT2Tokenizer, AddedToken
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [3]:
wav2vec_pretrained = "facebook/wav2vec2-base"
gpt_pretrained = "gpt2"

# Should aware that pad_token_id is used to compute CTC loss, 
# so pad_token configuration for both tokenizer and model should be the same
args = {
#     'pad_token': 'Ġ', 'pad_token_id': 220,
#     'unk_token': 'Ġ', 'unk_token_id': 220,
    'pad_token': "<|endoftext|>", 'pad_token_id': 50256,
    'unk_token': "<|endoftext|>", 'unk_token_id': 50256,
    'bos_token': "<|endoftext|>", 'bos_token_id': 50256,
    'eos_token': "<|endoftext|>", 'eos_token_id': 50256,
    
    'n_positions': 128,
    
    'add_adapter': True,
    'adapter_kernel_size': 6, 
    'adapter_stride': 2,
    'num_adapter_layers': 3,
}


config = Wav2Vec2GPTConfig(**args)

In [4]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(wav2vec_pretrained, 
                                                             cache_dir=model_cache_dir,
                                                             **args)

tokenizer = GPT2Tokenizer.from_pretrained(gpt_pretrained,
                                          cache_dir=model_cache_dir,
                                          **args)

In [5]:
model = Wav2Vec2GPTModel(config=config)

model.wav2vec2.from_pretrained(wav2vec_pretrained, cache_dir=model_cache_dir)
model.transformer.from_pretrained(gpt_pretrained, cache_dir=model_cache_dir)


# device_map = {
#     0: [0, 1, 2, 3, 4,],
#     2: [5, 6, 7, 8, 9, 10, 11, ],
# }
# model.gpt2lm.parallelize(device_map)


model.freeze_feature_extractor()
model.freeze_feature_projection()
# model.freeze_wav2vec_encoder() # not exists here
model.unfreeze_wav2vec_adapter()
model.unfreeze_rnn_compressor()
model.freeze_gpt_decoder()
model.unfreeze_lm_head()

/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model2: ['wav2vec2.encoder.layers.10.attention.k_proj.bias', 'wav2vec2.encoder.layers.2.attention.k_proj.weight', 'wav2vec2.encoder.layers.5.attention.v_proj.bias', 'wav2vec2.encoder.layers.9.attention.v_proj.bias', 'wav2vec2.encoder.layers.4.attention.k_proj.bias', 'wav2vec2.encoder.layers.4.attention.v_proj.bias', 'wav2vec2.encoder.layers.8.attention.out_proj.weight', 'wav2vec2.encoder.layers.6.feed_forward.intermediate_dense.weight', 'wav2vec2.encoder.layers.8.feed_forward.intermediate

In [6]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", 
                       split="validation", 
                       cache_dir=data_cache_dir
                      )

dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate
audio_inputs = [d["audio"]["array"] for d in dataset]

print(dataset, sampling_rate)

Reusing dataset librispeech_asr (/data4/yoomcache/datasets/hf-internal-testing___librispeech_asr/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)
Loading cached sorted indices for dataset at /data4/yoomcache/datasets/hf-internal-testing___librispeech_asr/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b/cache-2f7c0cbee6ef3aa1.arrow


Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
}) 16000


In [7]:
# text_inputs = dataset["text"]
from example.librispeech_asr_demo import text_inputs

In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_values, tokenized_output):
        self.input_values = input_values
        self.tokenized_output = tokenized_output

    def __getitem__(self, idx):
        item = dict()
        item['input_values'] = self.input_values['input_values'][idx]
        item['labels'] = self.tokenized_output['input_ids'][idx]
        item['output_attention_mask'] = self.tokenized_output['attention_mask'][idx]
        return item

    def __len__(self):
        return len(self.input_values['input_values'])

    
input_values = feature_extractor(audio_inputs, 
                                      sampling_rate=sampling_rate,
                                      return_tensors="pt",
                                      padding='longest',
                                     )

tokenized_output = tokenizer(text_inputs,
                             return_tensors="pt",
                             # padding='longest',
                             padding='max_length',
                             max_length=args['n_positions']
                         )

train_dataset = CustomDataset(input_values, tokenized_output)
# val_dataset = CustomDataset(input_values, tokenized_output)
# test_dataset = CustomDataset(input_values, tokenized_output)

In [9]:
# # load rouge for validation
# rouge = load_metric("rouge")

# def compute_metrics(pred):
#     labels_ids = pred.label_ids
#     pred_ids = pred.predictions

#     # all unnecessary tokens are removed
#     pred_str = decoder_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     labels_ids[labels_ids == -100] = decoder_tokenizer.eos_token_id
#     label_str = decoder_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

#     rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

#     return {
#         "rouge2_precision": round(rouge_output.precision, 4),
#         "rouge2_recall": round(rouge_output.recall, 4),
#         "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
#     }

In [10]:
batch_size = 3
steps_per_epoch = math.ceil(len(train_dataset) / batch_size)


# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
#     predict_with_generate=True,
    output_dir=os.path.join(checkpoint_dir, "wav2vec2gpt/unfreeze-rnn"),
    # do_train=True,
    # do_eval=False,
    # do_predict=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size * 5,
    learning_rate=1e-4, 
    weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0,
    num_train_epochs=1000,
    max_steps=-1,
    lr_scheduler_type='cosine', 
    # warmup_ratio=0.0, 
    
    logging_strategy='steps',
    save_strategy='steps',
    evaluation_strategy='steps',
    logging_steps=1 * steps_per_epoch,
    save_steps=2 * steps_per_epoch,
    eval_steps=1 * steps_per_epoch,
    warmup_steps=10 * steps_per_epoch,
    save_total_limit=10,
    overwrite_output_dir=True,
)

In [11]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
#     compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)


# start training
trainer.train()

/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 73
  Num Epochs = 1000
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 13000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/modules/rnn.py:942: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Tr

Step,Training Loss,Validation Loss
25,37.771000,104.160675
50,26.774500,60.313110
75,19.877300,37.778687
100,11.433800,24.680008
125,6.497600,20.343147
150,5.089400,16.175438
175,4.477200,14.517616
200,3.966500,14.067318
225,4.127000,13.748132
250,3.717800,13.503182


***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-50
Configuration saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-50/config.json
Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-50/pytorch_model.bin
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-100
Configuration saved in /data4/

***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-600
Configuration saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-600/config.json
Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-600/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-100] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 

Deleting older checkpoint [/data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-550] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-1100
Configuration saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-1100/config.json
Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-1100/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-600] due to args.save_total_limit
/home/y

...

Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-11500/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-11000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-11550
Configuration saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-11550/config.json
Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-11550/pytorch_model.bin
Deleting older c

***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12000
Configuration saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12000/config.json
Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-11500] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch

Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12450/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-11950] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12500
Configuration saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12500/config.json
Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12500/pytorch_model.bin
Deleting older c

***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
Saving model checkpoint to /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12950
Configuration saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12950/config.json
Model weights saved in /data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12950/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/wav2vec2gpt/unfreeze-rnn/checkpoint-12450] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 73
  Batch size = 30
***** Running Evaluation *****
  Num examples = 73
  Batch

TrainOutput(global_step=13000, training_loss=0.6328350240083841, metrics={'train_runtime': 7463.8848, 'train_samples_per_second': 9.78, 'train_steps_per_second': 1.742, 'total_flos': 2.3578709701075472e+19, 'train_loss': 0.6328350240083841, 'epoch': 1000.0})

In [12]:
wandb.finish()

eval/loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▄▃▁▄▄▁▁▃▂█▂▂▆▄▆▅▄▂▄▇▆▃▆▄█▄▂▂▃▇▁▅█▄▄▁▁▇▇▂
eval/samples_per_second,▅▆█▄▅██▅▇▁▇▆▃▅▃▄▅▇▅▂▂▆▂▅▁▄▇▇▆▂█▄▁▅▅██▂▂▇
eval/steps_per_second,▅▆█▄▅██▅▇▁▇▆▃▅▃▄▅▇▅▂▂▆▂▅▁▄▇▇▆▂█▄▁▅▅██▂▂▇
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,▇███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▆▃▃▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [13]:
##### example


BATCH_SIZE = 8
i = 3
device = 'cuda:0'


audio_batch = audio_inputs[i*BATCH_SIZE:i*BATCH_SIZE+BATCH_SIZE]
audio_feature_batch = feature_extractor(audio_batch, 
                                      sampling_rate=sampling_rate,
                                      return_tensors="pt",
                                      padding='longest',
                                     ).input_values
print(audio_feature_batch.size())


text_batch = text_inputs[i*BATCH_SIZE:i*BATCH_SIZE+BATCH_SIZE]

text_tokens_batch = tokenizer(text_batch, 
                              return_tensors="pt",
                              padding='max_length',
                              max_length=train_dataset.tokenized_output['input_ids'].shape[1]
                             )
print(text_tokens_batch['attention_mask'].size())

with torch.no_grad():
    audio_embedding = model(input_values=audio_feature_batch.to(device), 
                            labels=text_tokens_batch['input_ids'].to(device),
                            output_attention_mask=text_tokens_batch['attention_mask'].to(device),)
print(audio_embedding.logits.shape)

pred_ids = torch.argmax(audio_embedding.logits, axis=-1)
print(pred_ids.size())
print()

for idx in range(BATCH_SIZE):
    print(text_batch[idx])
    print(tokenizer.decode([key for key, _group in groupby(pred_ids[idx])]))
    print()

torch.Size([8, 143920])
torch.Size([8, 128])
torch.Size([8, 128, 50257])
torch.Size([8, 128])

"He doesn't work at all."
"He doesn't work at all."<|endoftext|>

In fact, there is nothing he can do in these dominions as well as our nomes, whose numbers are so great that it worries us to keep them all busy.
In fact, there is nothing can do in theseions as well as ouromes, whose numbers are numbers so great that it worries us to keep them all busy.<|endoftext|>

"Not exactly," returned Kaliko.
"Not exactly," returned Kal.<|endoftext|>

"Where is my brother now?"
"Where is my brother now?"<|endoftext|>

inquired Shaggy. "In the Metal Forest."
inquired Shaggy. "In the Metal Forest."<|endoftext|>

"Where is that?"
"Where is that?"<|endoftext|>

"The Metal Forest is in the Great Domed Cavern, the largest in all our dominions," replied Kaliko.
"The Metal Forest is in Great Domed Cavern the largest in all our domin," replied Kal.<|endoftext|>

Kaliko hesitated.
Kaliko hesitated.<|endoftext|>



In [14]:
# import IPython

# IPython.display.Audio(dataset[4]['audio']['path'])

### Wandb 149
#### `diff = output_from_RNN[:,:-1,-1] - output_from_RNN[:,1:,-1]`



"He doesn't work at all."
"He doesn't work at."<|endoftext|>

In fact, there is nothing he can do in these dominions as well as our nomes, whose numbers are so great that it worries us to keep them all busy.
In fact, is nothing he can do in these dominions as well as our, whose are so great that worries us to keep them all busy.<|endoftext|>

"Not exactly," returned Kaliko.
"Not exactly," returned Kaliko.<|endoftext|>

"Where is my brother now?"
"Where is my brother now?"<|endoftext|>

inquired Shaggy. "In the Metal Forest."
inquired Shaggy. "In the Metal Forest."<|endoftext|>

"Where is that?"
"Where is that?"<|endoftext|>

"The Metal Forest is in the Great Domed Cavern, the largest in all our dominions," replied Kaliko.
"The Metal is the Great Domed Cavern, and largest in all our dominions," replied Kaliko.<|endoftext|>

Kaliko hesitated.
Kaliko.<|endoftext|>


### Wandb 150
#### `diff = nn.ConstantPad2d((1, 0, 0, 0,), 0)(output_from_RNN[:,:-1,-1]) - output_from_RNN[:,:,-1]`

"He doesn't work at all."
"He doesn't work at all."<|endoftext|>

In fact, there is nothing he can do in these dominions as well as our nomes, whose numbers are so great that it worries us to keep them all busy.
In, there is nothing he can<|endoftext|> do in theseions<|endoftext|> as well as, n and, whose, whose numbers are so great that it worries us to them all busy.<|endoftext|>

"Not exactly," returned Kaliko.
"Not exactly," returned Kaliko.<|endoftext|>

"Where is my brother now?"
" is brother now?"<|endoftext|>

inquired Shaggy. "In the Metal Forest."
inquired Shaggy "<|endoftext|> the Metal Forest."<|endoftext|>

"Where is that?"
" is that<|endoftext|>

"The Metal Forest is in the Great Domed Cavern, the largest in all our dominions," replied Kaliko.
"The Metal Forest is in Great,ed Cavern the, the in all our domin," replied Kaliko.<|endoftext|>

Kaliko hesitated.
Kaliko hesitated.<|endoftext|>

### Wandb 151
#### `diff = output_from_RNN[:,:,-1] - nn.ConstantPad2d((0, 1, 0, 0,), 0)(output_from_RNN[:,1:,-1])`


"He doesn't work at all."
"He doesn't work at all."<|endoftext|>

In fact, there is nothing he can do in these dominions as well as our nomes, whose numbers are so great that it worries us to keep them all busy.
In fact, is he can in domin<|endoftext|> as well as our nomes, numbers are so that it us to keep them all busy.<|endoftext|>

"Not exactly," returned Kaliko.
"Not the exactly," returned Kal.<|endoftext|>

"Where is my brother now?"
"Where is my brother now?"<|endoftext|>

inquired Shaggy. "In the Metal Forest."
inquired Shaggy.In the Metal Forest<|endoftext|>

"Where is that?"
"Where is that?"<|endoftext|>

"The Metal Forest is in the Great Domed Cavern, the largest in all our dominions," replied Kaliko.
"The Metal is in the Great Domed, the in all our dominions," Kaliko<|endoftext|>

Kaliko hesitated.
Kaliko hesitated.<|endoftext|>
